In [1]:
import numpy as np
import h5py

import e2c as e2c_util

# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# The GPU id to use, usually either "0" or "1"

import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"

from keras import backend as K
from keras.layers import Input
from keras.models import Model
from keras.optimizers import Adam
from keras import losses

Using TensorFlow backend.


In [2]:
def reconstruction_loss(x, t_decoded):
    '''Reconstruction loss for the plain VAE'''
    v = 0.1
    # return K.mean(K.sum((K.batch_flatten(x) - K.batch_flatten(t_decoded)) ** 2 / (2*v) + 0.5*K.log(2*np.pi*v), axis=-1))
    return K.mean(K.sum((K.batch_flatten(x) - K.batch_flatten(t_decoded)) ** 2 / (2*v), axis=-1))
    # return K.sum((K.batch_flatten(x) - K.batch_flatten(t_decoded)) ** 2, axis=-1)


def kl_normal_loss(qm, q_logv, pm, p_logv):
    # 0.5 * (torch.log(pv) - torch.log(qv) + qv / pv + (qm - pm).pow(2) / pv - 1)
    # -0.5 * K.sum(1 + t_log_var - K.square(t_mean) - K.exp(t_log_var), axis=-1)
    kl = -0.5 * (1 - p_logv + q_logv - K.exp(q_logv) / K.exp(p_logv) - K.square(qm - pm) / K.exp(p_logv))
    return K.mean(K.sum(kl, axis=-1))


def get_flux_loss(m, state, state_pred):
    # state, state_pred shape (batch_size, 60, 60, 2)
    # p, p_pred shape (batch_size, 60, 60, 1)
    # k shape (batch_size, 60, 60, 1)
    
    # Only consider discrepancies in total flux, not in phases (saturation not used) 
    
    perm = K.exp(m)
    p = K.expand_dims(state[:, :, :, 1], -1)
    p_pred = K.expand_dims(state_pred[:, :, :, 1], -1)

    #print(K.in_shape(xxx))
    
    tran_x = 1./perm[:, 1:, ...] + 1./perm[:, :-1, ...]
    tran_y = 1./perm[:, :, 1:, ...] + 1./perm[:, :, :-1, ...]
    flux_x = (p[:, 1:, ...] - p[:, :-1, ...]) / tran_x
    flux_y = (p[:, :, 1:, :] - p[:, :, :-1, :]) / tran_y
    flux_x_pred = (p_pred[:, 1:, ...] - p_pred[:, :-1, ...]) / tran_x
    flux_y_pred = (p_pred[:, :, 1:, :] - p_pred[:, :, :-1, :]) / tran_y

    loss_x = K.sum(K.abs(K.batch_flatten(flux_x) - K.batch_flatten(flux_x_pred)), axis=-1)
    loss_y = K.sum(K.abs(K.batch_flatten(flux_y) - K.batch_flatten(flux_y_pred)), axis=-1)

    loss_flux = K.mean(loss_x + loss_y)
    return loss_flux

In [3]:
def get_binary_sat_loss(state, state_pred):
    
    sat_threshold = 0.105
    sat = K.expand_dims(state[:, :, :, 0], -1)
    sat_pred = K.expand_dims(state_pred[:, :, :, 0], -1)
    
    
    sat_bool = K.greater_equal(sat, sat_threshold) #will return boolean values
    sat_bin = K.cast(sat_bool, dtype=K.floatx()) #will convert bool to 0 and 1  
    
    sat_pred_bool = K.greater_equal(sat_pred, sat_threshold) #will return boolean values
    sat_pred_bin = K.cast(sat_pred_bool, dtype=K.floatx()) #will convert bool to 0 and 1  
    
#     binary_loss = K.sum(K.abs(K.batch_flatten(sat_bin) - K.batch_flatten(sat_pred_bin)), axis=-1)
    
    binary_loss = losses.binary_crossentropy(sat_bin, sat_pred_bin)
    return K.mean(binary_loss)

In [4]:
def create_e2c(latent_dim, u_dim, input_shape):
    '''
    Creates a E2C.

    Args:
        latent_dim: dimensionality of latent space
        return_kl_loss_op: whether to return the operation for
                           computing the KL divergence loss.

    Returns:
        The VAE model. If return_kl_loss_op is True, then the
        operation for computing the KL divergence loss is
        additionally returned.
    '''

    encoder_ = e2c_util.create_encoder(latent_dim, input_shape)
    decoder_ = e2c_util.create_decoder(latent_dim, input_shape)
    transition_ = e2c_util.create_trans(latent_dim, u_dim)
    sampler_ = e2c_util.create_sampler()

    return encoder_, decoder_, transition_, sampler_

In [5]:
# Create plain E2C model and associated loss operations

################### case specification ######################

#     data_dir = '/data/cees/zjin/lstm_rom/datasets/9W_BHP/'
data_dir = '/data/cees/zjin/lstm_rom/datasets/9W_BHP_RATE/'
output_dir = '/data3/Astro/lstm_rom/e2c_larry/saved_models/'

#     case_name = '9w_bhp'
case_name = '9w_bhp_rate'

#     case_suffix = '_single_out_rel_2'
case_suffix = '_fix_wl_rel_1'
#     case_suffix = '_single_out_rel_3'
train_suffix = '_with_p'
model_suffix = '_flux_loss'


train_file = case_name + '_e2c_train' + case_suffix + train_suffix + '_n6600_dt20day_nt22_nrun300.mat'
eval_file = case_name + '_e2c_eval' + case_suffix + train_suffix +'_n2200_dt20day_nt22_nrun100.mat'

#################### model specification ##################
epoch = 10
batch_size = 4
learning_rate = 1e-4
latent_dim = 50
u_dim = 9 # control dimension

# load data
hf_r = h5py.File(data_dir + train_file, 'r')
state_t_train = np.array(hf_r.get('state_t'))
state_t1_train = np.array(hf_r.get('state_t1'))
bhp_train = np.array(hf_r.get('bhp'))
hf_r.close()

num_train = state_t_train.shape[0]

hf_r = h5py.File(data_dir + eval_file, 'r')
state_t_eval = np.array(hf_r.get('state_t'))
state_t1_eval = np.array(hf_r.get('state_t1'))
bhp_eval = np.array(hf_r.get('bhp'))
hf_r.close()

m = np.loadtxt("/data/cees/zjin/lstm_rom/sim_runs/case4_9w_bhp_rate/template/logk1.dat")
m = m.reshape(60,60,1)
print('m shape is ', m.shape)
#     m_tf = K.placeholder((batch_size, 60, 60 ,1))
m_tf = Input(shape=(60, 60, 1))



m_eval = np.repeat(np.expand_dims(m, axis = 0), state_t_eval.shape[0], axis = 0)
print("m_eval shape is ", m_eval.shape)
m = np.repeat(np.expand_dims(m,axis = 0), state_t_train.shape[0], axis = 0)
print("m shape is ", m.shape)


m shape is  (60, 60, 1)
m_eval shape is  (2200, 60, 60, 1)
m shape is  (6600, 60, 60, 1)


In [8]:

# Construct E2C
input_shape = (60, 60, 2)
encoder, decoder, transition, sampler = create_e2c(latent_dim, u_dim, input_shape)

xt = Input(shape=input_shape)
xt1 = Input(shape=input_shape)
ut = Input(shape=(u_dim, ))

zt_mean, zt_logvar = encoder(xt)
zt = sampler([zt_mean, zt_logvar])
xt_rec = decoder(zt)

zt1_mean, zt1_logvar = encoder(xt1)

# zt1_pred, zt1_mean_pred, zt1_logvar_pred = transition([zt, ut])
zt1_pred, zt1_mean_pred = transition([zt, zt_mean, ut])
xt1_pred = decoder(zt1_pred)

# Compute loss
loss_rec_t = reconstruction_loss(xt, xt_rec)
loss_rec_t1 = reconstruction_loss(xt1, xt1_pred)

loss_flux_t = get_flux_loss(m_tf, xt, xt_rec) / 1000.
loss_flux_t1 = get_flux_loss(m_tf, xt1, xt1_pred) / 1000.

binary_sat_loss_t = get_binary_sat_loss(xt, xt_rec) * 20
binary_sat_loss_t1 = get_binary_sat_loss(xt1, xt1_pred) * 20


loss_kl = kl_normal_loss(zt_mean, zt_logvar, 0., 0.)  # log(1.) = 0.
# loss_bound = loss_rec_t + loss_rec_t1 + loss_kl  + loss_flux_t + loss_flux_t1 + binary_sat_loss_t + binary_sat_loss_t1
loss_bound = loss_rec_t + loss_rec_t1 + loss_kl + binary_sat_loss_t + binary_sat_loss_t1

# loss_trans = kl_normal_loss(zt1_mean_pred, zt1_logvar_pred, zt1_mean, zt1_logvar)

# Use zt_logvar to approximate zt1_logvar_pred
loss_trans = kl_normal_loss(zt1_mean_pred, zt_logvar, zt1_mean, zt1_logvar)


trans_loss_weight = 1.0 # lambda in E2C paper Eq. (11)
loss = loss_bound + trans_loss_weight * loss_trans



In [9]:
# Optimization
opt = Adam(lr=learning_rate)

trainable_weights = encoder.trainable_weights + decoder.trainable_weights + transition.trainable_weights

updates = opt.get_updates(loss, trainable_weights)

iterate = K.function([xt, ut, xt1, m_tf], [loss, loss_rec_t, loss_rec_t1, loss_kl, loss_trans, loss_flux_t, loss_flux_t1, binary_sat_loss_t, binary_sat_loss_t1], updates=updates)

eval_loss = K.function([xt, ut, xt1, m_tf], [loss])

num_batch = int(num_train/batch_size)

for e in range(epoch):
    for ib in range(num_batch):
        ind0 = ib * batch_size
        state_t_batch = state_t_train[ind0:ind0+batch_size, ...]
        state_t1_batch = state_t1_train[ind0:ind0 + batch_size, ...]
        bhp_batch = bhp_train[ind0:ind0 + batch_size, ...]
        m_batch = m[ind0:ind0 + batch_size, ...]

        output = iterate([state_t_batch, bhp_batch, state_t1_batch, m_batch])

        # tf.session.run(feed_dict={xt: sat_t_batch, ut: bhp_batch, xt1: sat_t1_batch}, ...
        #                fetches= [loss, loss_rec_t, loss_rec_t1, loss_kl, loss_trans, updates])
        # But output tensor for the updates operation is not returned

        if ib % 10 == 0:
            print('Epoch %d/%d, Batch %d/%d, Loss %f, Loss rec %f, loss rec t1 %f, loss kl %f, loss_trans %f, loss flux %f, loss flux t1 %f, binary loss %f, binary loss t1 %f'
                  % (e+1, epoch, ib+1, num_batch, output[0], output[1], output[2], output[3], output[4], output[5], output[6], output[7], output[8]))
    eval_loss_val = eval_loss([state_t_eval, bhp_eval, state_t1_eval, m_eval])

    print('Epoch %d/%d, Train loss %f, Eval loss %f' % (e + 1, epoch, output[0], eval_loss_val[0]))


encoder.save_weights(output_dir + 'e2c_encoder_' + case_name + case_suffix + train_suffix + model_suffix + '_nt%d_l%d_lr%.0e_ep%d.h5' \
                     % (num_train, latent_dim, learning_rate, epoch))
decoder.save_weights(output_dir + 'e2c_decoder_' + case_name + case_suffix + train_suffix + model_suffix + '_nt%d_l%d_lr%.0e_ep%d.h5' \
                     % (num_train, latent_dim, learning_rate, epoch))
transition.save_weights(output_dir + 'e2c_transition_' + case_name + case_suffix + train_suffix + model_suffix + '_nt%d_l%d_lr%.0e_ep%d.h5' \
                        % (num_train, latent_dim, learning_rate, epoch))

Epoch 1/10, Batch 1/1650, Loss 3994.166504, Loss rec 1699.668335, loss rec t1 2089.065918, loss kl 0.009152, loss_trans 0.006770, loss flux 35.480938, loss flux t1 38.295868, binary loss 82.045586, binary loss t1 123.370590
Epoch 1/10, Batch 11/1650, Loss 1711.158936, Loss rec 638.012512, loss rec t1 775.507568, loss kl 5.425941, loss_trans 4.968709, loss flux 192.247787, loss flux t1 185.470734, binary loss 163.482285, binary loss t1 123.761917
Epoch 1/10, Batch 21/1650, Loss 1258.066162, Loss rec 452.084656, loss rec t1 467.091461, loss kl 5.420256, loss_trans 4.436678, loss flux 204.095932, loss flux t1 201.001984, binary loss 175.654083, binary loss t1 153.379013
Epoch 1/10, Batch 31/1650, Loss 1027.259277, Loss rec 325.748077, loss rec t1 344.808136, loss kl 3.507523, loss_trans 3.334332, loss flux 193.965866, loss flux t1 191.596573, binary loss 188.391708, binary loss t1 161.469482
Epoch 1/10, Batch 41/1650, Loss 1001.368164, Loss rec 289.152985, loss rec t1 375.389893, loss kl 

Epoch 1/10, Batch 371/1650, Loss 284.475647, Loss rec 72.454689, loss rec t1 76.818710, loss kl 2.516236, loss_trans 1.333847, loss flux 168.147888, loss flux t1 173.684692, binary loss 65.589104, binary loss t1 65.763069
Epoch 1/10, Batch 381/1650, Loss 224.523529, Loss rec 61.518127, loss rec t1 70.384827, loss kl 2.144515, loss_trans 1.093054, loss flux 166.211823, loss flux t1 167.397049, binary loss 47.811848, binary loss t1 41.571159
Epoch 1/10, Batch 391/1650, Loss 360.215912, Loss rec 73.209908, loss rec t1 82.546211, loss kl 2.401870, loss_trans 1.307721, loss flux 166.333282, loss flux t1 177.086777, binary loss 105.335327, binary loss t1 95.414894
Epoch 1/10, Batch 401/1650, Loss 251.751511, Loss rec 71.430481, loss rec t1 75.874115, loss kl 1.618123, loss_trans 0.825736, loss flux 155.069260, loss flux t1 144.989975, binary loss 50.338249, binary loss t1 51.664833
Epoch 1/10, Batch 411/1650, Loss 230.476028, Loss rec 63.455208, loss rec t1 70.713493, loss kl 2.565700, loss_

Epoch 1/10, Batch 741/1650, Loss 184.266876, Loss rec 34.562340, loss rec t1 39.742981, loss kl 1.238271, loss_trans 0.596800, loss flux 125.674194, loss flux t1 127.396774, binary loss 56.751930, binary loss t1 51.374546
Epoch 1/10, Batch 751/1650, Loss 207.357773, Loss rec 35.141182, loss rec t1 34.695145, loss kl 1.270299, loss_trans 0.550329, loss flux 116.872383, loss flux t1 114.924538, binary loss 68.982224, binary loss t1 66.718597
Epoch 1/10, Batch 761/1650, Loss 277.755432, Loss rec 41.972031, loss rec t1 47.764313, loss kl 1.529584, loss_trans 0.621046, loss flux 122.735069, loss flux t1 119.983429, binary loss 100.948730, binary loss t1 84.919731
Epoch 1/10, Batch 771/1650, Loss 128.029312, Loss rec 33.268806, loss rec t1 34.882355, loss kl 0.939799, loss_trans 0.522696, loss flux 105.936218, loss flux t1 108.293770, binary loss 30.565823, binary loss t1 27.849831
Epoch 1/10, Batch 781/1650, Loss 179.820663, Loss rec 24.643578, loss rec t1 28.682842, loss kl 0.954843, loss_

Epoch 1/10, Batch 1111/1650, Loss 131.879837, Loss rec 13.253901, loss rec t1 12.442616, loss kl 0.671845, loss_trans 0.240865, loss flux 87.195175, loss flux t1 91.815605, binary loss 51.283779, binary loss t1 53.986832
Epoch 1/10, Batch 1121/1650, Loss 139.285843, Loss rec 24.742865, loss rec t1 21.517204, loss kl 1.254977, loss_trans 0.471608, loss flux 112.613457, loss flux t1 114.577271, binary loss 49.351860, binary loss t1 41.947334
Epoch 1/10, Batch 1131/1650, Loss 107.282700, Loss rec 19.994045, loss rec t1 21.114084, loss kl 1.801281, loss_trans 0.575744, loss flux 122.413048, loss flux t1 120.702019, binary loss 33.725258, binary loss t1 30.072281
Epoch 1/10, Batch 1141/1650, Loss 131.763657, Loss rec 17.790386, loss rec t1 20.260170, loss kl 1.181509, loss_trans 0.482798, loss flux 112.615486, loss flux t1 111.256577, binary loss 47.342220, binary loss t1 44.706570
Epoch 1/10, Batch 1151/1650, Loss 91.749977, Loss rec 12.804768, loss rec t1 15.063910, loss kl 1.387040, loss

Epoch 1/10, Batch 1481/1650, Loss 79.112968, Loss rec 13.771149, loss rec t1 12.356915, loss kl 0.846596, loss_trans 0.240342, loss flux 101.137665, loss flux t1 104.656197, binary loss 25.917114, binary loss t1 25.980854
Epoch 1/10, Batch 1491/1650, Loss 94.276962, Loss rec 14.006546, loss rec t1 12.463839, loss kl 1.281126, loss_trans 0.310988, loss flux 112.275780, loss flux t1 111.829376, binary loss 35.078377, binary loss t1 31.136084
Epoch 1/10, Batch 1501/1650, Loss 137.513077, Loss rec 15.465912, loss rec t1 19.369797, loss kl 1.159825, loss_trans 0.346162, loss flux 108.043503, loss flux t1 109.580696, binary loss 52.013496, binary loss t1 49.157883
Epoch 1/10, Batch 1511/1650, Loss 107.152878, Loss rec 17.309219, loss rec t1 19.116594, loss kl 0.702352, loss_trans 0.284940, loss flux 95.441689, loss flux t1 94.396362, binary loss 35.224159, binary loss t1 34.515610
Epoch 1/10, Batch 1521/1650, Loss 88.277916, Loss rec 8.457202, loss rec t1 9.076288, loss kl 0.751772, loss_tra

...

Epoch 9/10, Batch 1571/1650, Loss 23.064871, Loss rec 2.002920, loss rec t1 2.188213, loss kl 0.334594, loss_trans 0.039908, loss flux 46.562943, loss flux t1 49.740726, binary loss 9.193896, binary loss t1 9.305340
Epoch 9/10, Batch 1581/1650, Loss 21.297260, Loss rec 2.413028, loss rec t1 2.659765, loss kl 0.161081, loss_trans 0.024003, loss flux 43.862202, loss flux t1 44.541283, binary loss 8.528963, binary loss t1 7.510422
Epoch 9/10, Batch 1591/1650, Loss 23.026283, Loss rec 2.150162, loss rec t1 1.930486, loss kl 0.259304, loss_trans 0.033808, loss flux 46.763641, loss flux t1 48.450169, binary loss 9.547684, binary loss t1 9.104839
Epoch 9/10, Batch 1601/1650, Loss 25.106018, Loss rec 4.048600, loss rec t1 5.687808, loss kl 0.103118, loss_trans 0.025201, loss flux 44.022938, loss flux t1 44.873569, binary loss 6.840787, binary loss t1 8.400503
Epoch 9/10, Batch 1611/1650, Loss 35.057575, Loss rec 2.825616, loss rec t1 3.816493, loss kl 0.136778, loss_trans 0.030166, loss flux 4

Epoch 10/10, Batch 301/1650, Loss 19.419838, Loss rec 2.991775, loss rec t1 3.470415, loss kl 0.177502, loss_trans 0.023663, loss flux 44.713478, loss flux t1 46.093834, binary loss 6.600152, binary loss t1 6.156332
Epoch 10/10, Batch 311/1650, Loss 44.031250, Loss rec 2.981250, loss rec t1 3.936551, loss kl 0.208498, loss_trans 0.037208, loss flux 45.368931, loss flux t1 47.468262, binary loss 17.958303, binary loss t1 18.909441
Epoch 10/10, Batch 321/1650, Loss 43.782967, Loss rec 3.748236, loss rec t1 4.546251, loss kl 0.238307, loss_trans 0.038939, loss flux 49.153862, loss flux t1 51.111305, binary loss 17.494781, binary loss t1 17.716448
Epoch 10/10, Batch 331/1650, Loss 27.653946, Loss rec 5.172676, loss rec t1 6.237511, loss kl 0.167707, loss_trans 0.027817, loss flux 44.978233, loss flux t1 46.069195, binary loss 8.103691, binary loss t1 7.944546
Epoch 10/10, Batch 341/1650, Loss 46.496628, Loss rec 7.114451, loss rec t1 7.646308, loss kl 0.285967, loss_trans 0.054810, loss fl

Epoch 10/10, Batch 681/1650, Loss 25.966505, Loss rec 1.793184, loss rec t1 2.476514, loss kl 0.187392, loss_trans 0.033364, loss flux 44.253761, loss flux t1 46.761341, binary loss 10.594224, binary loss t1 10.881828
Epoch 10/10, Batch 691/1650, Loss 30.396088, Loss rec 2.893751, loss rec t1 3.284925, loss kl 0.338326, loss_trans 0.040341, loss flux 51.320744, loss flux t1 53.161373, binary loss 12.240067, binary loss t1 11.598677
Epoch 10/10, Batch 701/1650, Loss 28.204744, Loss rec 2.990904, loss rec t1 3.233868, loss kl 0.317570, loss_trans 0.035994, loss flux 48.966656, loss flux t1 49.252121, binary loss 11.124416, binary loss t1 10.501994
Epoch 10/10, Batch 711/1650, Loss 24.319387, Loss rec 4.231067, loss rec t1 4.727338, loss kl 0.443621, loss_trans 0.057124, loss flux 54.310509, loss flux t1 56.846996, binary loss 7.263197, binary loss t1 7.597039
Epoch 10/10, Batch 721/1650, Loss 33.844528, Loss rec 2.492548, loss rec t1 3.251204, loss kl 0.286484, loss_trans 0.037231, loss 

Epoch 10/10, Batch 1061/1650, Loss 31.732777, Loss rec 6.119087, loss rec t1 6.694645, loss kl 0.283970, loss_trans 0.044811, loss flux 51.159588, loss flux t1 52.576092, binary loss 9.259722, binary loss t1 9.330542
Epoch 10/10, Batch 1071/1650, Loss 30.933632, Loss rec 3.378658, loss rec t1 4.086647, loss kl 0.307067, loss_trans 0.046130, loss flux 46.469769, loss flux t1 49.911594, binary loss 11.049026, binary loss t1 12.066102
Epoch 10/10, Batch 1081/1650, Loss 28.568359, Loss rec 5.669531, loss rec t1 5.901770, loss kl 0.235097, loss_trans 0.036658, loss flux 47.609249, loss flux t1 48.170361, binary loss 8.794426, binary loss t1 7.930880
Epoch 10/10, Batch 1091/1650, Loss 29.487293, Loss rec 3.207216, loss rec t1 3.433768, loss kl 0.147853, loss_trans 0.030807, loss flux 42.918709, loss flux t1 45.259674, binary loss 11.158516, binary loss t1 11.509131
Epoch 10/10, Batch 1101/1650, Loss 28.828339, Loss rec 2.312356, loss rec t1 2.611082, loss kl 0.190459, loss_trans 0.034176, lo

Epoch 10/10, Batch 1441/1650, Loss 28.391697, Loss rec 2.100214, loss rec t1 2.684272, loss kl 0.339705, loss_trans 0.050030, loss flux 46.276672, loss flux t1 50.148605, binary loss 12.097208, binary loss t1 11.120268
Epoch 10/10, Batch 1451/1650, Loss 21.815285, Loss rec 2.658785, loss rec t1 2.638445, loss kl 0.241684, loss_trans 0.042897, loss flux 46.146160, loss flux t1 46.905964, binary loss 8.626740, binary loss t1 7.606735
Epoch 10/10, Batch 1461/1650, Loss 21.119450, Loss rec 1.481739, loss rec t1 1.447634, loss kl 0.202139, loss_trans 0.037621, loss flux 44.691841, loss flux t1 45.481277, binary loss 8.864203, binary loss t1 9.086114
Epoch 10/10, Batch 1471/1650, Loss 30.843189, Loss rec 2.220883, loss rec t1 2.481616, loss kl 0.260002, loss_trans 0.032680, loss flux 48.475510, loss flux t1 49.480202, binary loss 13.135208, binary loss t1 12.712798
Epoch 10/10, Batch 1481/1650, Loss 17.801043, Loss rec 2.783973, loss rec t1 2.847620, loss kl 0.223592, loss_trans 0.029069, lo